# Import

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive, spark
from wmfdata.utils import print_err, pd_display_all

# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



# Preparation

In [3]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month),
    #"metrics_prev_month": str(metrics_month - 1),
    #"metrics_month_start": str(metrics_month.start_time), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    #"metrics_month_end": str(last_month),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

# MariaDB and Hive query metrics


In [7]:
# Code to Suppress PySpark Warning messages
import warnings
warnings.filterwarnings('ignore') 

In [8]:
# Code to Suppress PySpark Warning messages

from IPython.display import HTML
HTML('''<script>
var code_show_err = false; 
var code_toggle_err = function() {
 var stderrNodes = document.querySelectorAll('[data-mime-type="application/vnd.jupyter.stderr"]')
 var stderr = Array.from(stderrNodes)
 if (code_show_err){
     stderr.forEach(ele => ele.style.display = 'block');
 } else {
     stderr.forEach(ele => ele.style.display = 'none');
 }
 code_show_err = !code_show_err
} 
document.addEventListener('DOMContentLoaded', code_toggle_err);
</script>
To toggle on/off output_stderr, click <a onclick="javascript:code_toggle_err()">here</a>.''')

In [9]:
queries = {
    "pageviews": {
        "file": "queries/pageviews.hql",
        "engine": "hive"
    },
    
    "automated_pageviews": {
        "file": "queries/automated_pageviews.hql",
        "engine": "hive"
    },
    "page_previews": {
        "file": "queries/page_previews.hql",
        "engine": "hive"
    },
    "unique_devices": {
        "file": "queries/unique_devices.hql",
        "engine": "hive"
    },
    "global_south_pageviews": {
        "file": "queries/global_south_pageviews.hql",
        "engine": "hive"
    },
    
    "global_south_previews": {
        "file": "queries/global_south_previews.hql",
        "engine": "hive"
    }
    
# Removing since mobile-heavy metrics are no longer tracked/reported on in the key product metrics presentations
    #"mobile-heavy_pageviews": {
    #    "file": "queries/mobile_heavy_pageviews.hql",
    #    "engine": "hive" }, 
    #"mobile-heavy_previews": {
    #   "file": "queries/mobile_heavy_previews.hql",
    #   "engine": "hive" }
    
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = spark.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running automated_pageviews on hive...                                          
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running page_previews on hive...                                                
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running unique_devices on hive...                                               
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_pageviews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.
Running global_south_previews on hive...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


# Combining and saving metrics

In [10]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

# Add Metrics for Interactions
new_metrics['interactions'] = new_metrics.apply(lambda x: x['previews_seen'] + x['total_pageview'], axis=1)
new_metrics['gs_interactions'] = new_metrics.apply(lambda x: x['gs_previews'] + x['gs_pageviews'], axis=1)

# Removing since mobile-heavy metrics are no longer tracked/reported on in the key product metrics presentations
# new_metrics['mh_interactions'] = new_metrics.apply(lambda x: x['mh_previews'] + x['mh_pageviews'], axis=1)


In [11]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,automated_pageviews,desktop,gs_interactions,gs_pageviews,gs_previews,interactions,mh_interactions,mh_pageviews,mh_previews,mobileweb,previews_seen,total_pageview,unique_devices
month,,,,,,,,,,,,,
2021-09-01,9.116296e+08,6.293941e+09,4.342242e+09,3.949492e+09,392749815.0,1.782378e+10,821283216.0,788543740.0,32739476.0,9.454794e+09,1.812547e+09,1.601124e+10,1.690497e+09
2021-10-01,9.011588e+08,6.513036e+09,4.356292e+09,3.973495e+09,382796754.0,1.836003e+10,NaN,NaN,NaN,9.701242e+09,1.872864e+09,1.648717e+10,1.837564e+09
2021-11-01,8.810505e+08,6.420327e+09,4.247671e+09,3.868070e+09,379601677.0,1.776851e+10,NaN,NaN,NaN,9.209694e+09,1.880481e+09,1.588803e+10,1.730051e+09
2021-12-01,9.556163e+08,5.997305e+09,4.053082e+09,3.699154e+09,353928149.0,1.762913e+10,NaN,NaN,NaN,9.545437e+09,1.802476e+09,1.582665e+10,1.728499e+09
2022-01-01,1.003627e+09,6.657032e+09,4.237581e+09,3.860724e+09,376857667.0,1.940130e+10,NaN,NaN,NaN,1.037124e+10,2.058830e+09,1.734247e+10,1.775062e+09
2022-02-01,9.794070e+08,6.381499e+09,4.110316e+09,3.767028e+09,343288151.0,1.787409e+10,NaN,NaN,NaN,9.383266e+09,1.833566e+09,1.604052e+10,1.827453e+09
2022-03-01,1.000023e+09,6.820776e+09,4.407957e+09,4.051236e+09,356720322.0,1.895002e+10,NaN,NaN,NaN,9.870969e+09,1.967056e+09,1.698296e+10,1.957130e+09
2022-04-01,1.257069e+09,6.517583e+09,4.023481e+09,3.700706e+09,322775480.0,1.793078e+10,NaN,NaN,NaN,9.357140e+09,1.773847e+09,1.615693e+10,1.878851e+09
2022-05-01,1.491689e+09,6.701587e+09,4.102891e+09,3.771585e+09,331305888.0,1.817494e+10,NaN,NaN,NaN,9.402734e+09,1.788005e+09,1.638694e+10,1.948874e+09


In [12]:
metrics.to_csv(FILENAME, sep="\t")